In [1]:
# iPyTest allows us to solve AoC using test-driven development principals.
import ipytest
ipytest.autoconfig(addopts=['--color=no'])

In [91]:
! pip install scipy

     ---------------------------------------- 39.9/39.9 MB 1.6 MB/s eta 0:00:00
  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [96]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path

ModuleNotFoundError: No module named 'scipy'

In [105]:
%%ipytest

def read_puzzle_input(puzzle_input):
    if not os.path.exists(puzzle_input):
        return

    with open(puzzle_input) as ff:
        dd = [ xx.strip() for xx in ff.readlines() ]

    rows = len(dd)
    cols = len(dd[0])
    data = np.ndarray(shape=(rows, cols), dtype=np.int32)

    start = None
    goal = None
    # convert letters to numbers
    for ii, row in enumerate(dd):
        for jj, val in enumerate(row):
            if val == 'S':
                start = (ii, jj)
                data[ii, jj] = 0
            elif val == 'E':
                goal = (ii, jj)
                data[ii, jj] = 26
            else:
                data[ii, jj] = ord(val) - ord('a')

    return data, start, goal


def test_read_puzzle_input():
    data, start, goal = read_puzzle_input(os.path.join("..", "dat", "day12_test.txt"))
    assert start == (0, 0)
    assert goal == (2, 5)
    assert data.shape == (5, 8)
    assert data[0, 7] == ord('m') - 97
    assert data[4, 0] == ord('a') - 97
    assert data[4, 7] == ord('i') - 97

def adjacents(data, pos):
    curheight = data[pos]
    for ii in range(-1, 2):
        for jj in range(-1, 2):
            rr = pos[0] + ii
            cc = pos[1] + jj


            if ii == 0 and jj == 0:
                # skip over ourselves
                continue
            elif ii != 0 and jj != 0:
                # don't look diag
                continue
            elif rr < 0 or cc < 0:
                # outside of bounds
                continue
            elif rr >= data.shape[0] or cc >= data.shape[1]:
                # outside of bounds
                continue
            elif data[rr, cc] > curheight + 1:
                # too high
                continue

            yield (rr, cc), data[rr, cc]

def test_adjacents():
    data, _, _ = read_puzzle_input(os.path.join("..", "dat", "day12_test.txt"))
    adj = list(adjacents(data, (0,0)))
    assert len(adj) == 2
    assert adj[0][0] == (0, 1)
    assert adj[0][1] == 0
    assert adj[1][0] == (1, 0)
    assert adj[1][1] == 0

    adj = list(adjacents(data, (4,7)))
    assert len(adj) == 2
    assert adj[0][0] == (3, 7)
    assert adj[0][1] == ord('j') - 97
    assert adj[1][0] == (4, 6)
    assert adj[1][1] == ord('h') - 97

    adj = list(adjacents(data, (1,1)))
    assert len(adj) == 4

    adj = list(adjacents(data, (0,2)))
    assert len(adj) == 2
    

======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-7.2.0, pluggy-1.0.0
rootdir: c:\Users\MichaelIhde\Git\advent-of-code-2022\python
collected 2 items

t_58129cfc90f04738840380712e605c16.py ..                                                     [100%]

======================================== 2 passed in 0.01s ========================================


In [106]:
%%ipytest


def part1(puzzle_input):
    data, start, goal = read_puzzle_input(puzzle_input)

    visited = set()
    parentsMap = {}
    pq = []

    nodeCosts = collections.defaultdict(lambda: float('inf'))
    nodeCosts[start] = 0

    heapq.heappush(pq, (0, start))

    while pq:
        _, curpos = heapq.heappop(pq)
        visited.add(curpos)

        if curpos == goal:
            break

        curheight = data[curpos]
        
        for adjpos, adjheight in adjacents(data, curpos):
            if adjpos in visited:
                continue

            parentsMap[adjpos] = curpos
            nodeCosts[adjpos] = 1
            heapq.heappush(pq, (1, adjpos))

    path = []
    curpos = goal
    while curpos != start:
        path.append(curpos)
        nextpos = parentsMap[curpos]
        curpos = nextpos

    return path, parentsMap, nodeCosts

def test_part1():
    path, parentsMap, nodeCosts = part1(os.path.join("..", "dat", "day12_test.txt"))
    assert len(path) == 31
       
path, parentsMap, nodeCosts = part1(os.path.join("..", "dat", "day12.txt"))
print(len(path))

KeyboardInterrupt: 

======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-7.2.0, pluggy-1.0.0
rootdir: c:\Users\MichaelIhde\Git\advent-of-code-2022\python
collected 1 item

t_58129cfc90f04738840380712e605c16.py .                                                      [100%]

======================================== 1 passed in 0.01s ========================================


In [111]:
%%ipytest

def bfs(graph_to_search, start, end):
    queue = [[start]]
    visited = set()

    while queue:
        # Gets the first path in the queue
        path = queue.pop(0)

        # Gets the last node in the path
        vertex = path[-1]

        # Checks if we got to the end
        if vertex == end:
            return path
        # We check if the current node is already in the visited nodes set in order not to recheck it
        elif vertex not in visited:
            # enumerate all adjacent nodes, construct a new path and push it into the queue
            for current_neighbour in graph_to_search.get(vertex, []):
                new_path = list(path)
                new_path.append(current_neighbour)
                queue.append(new_path)

            # Mark the vertex as visited
            visited.add(vertex)

def part1(puzzle_input):
    data, start, goal = read_puzzle_input(puzzle_input)

    graph = {}
    for ii in range(data.shape[0]):
        for jj in range(data.shape[1]):
            curpos = (ii, jj)
            graph.setdefault((ii, jj), []).extend([ x[0] for x in adjacents(data, curpos) ])

    #pprint.pprint(graph)
                
    return bfs(graph, start, goal)

def test_part1():
    path = part1(os.path.join("..", "dat", "day12_test.txt"))
    assert len(path) == 32
       
path = part1(os.path.join("..", "dat", "day12.txt"))
print(len(path)-1)

437
======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-7.2.0, pluggy-1.0.0
rootdir: c:\Users\MichaelIhde\Git\advent-of-code-2022\python
collected 1 item

t_58129cfc90f04738840380712e605c16.py .                                                      [100%]

======================================== 1 passed in 0.01s ========================================


In [123]:
%%ipytest

def bfs(graph_to_search, start, end):
    queue = [[start]]
    visited = set()

    while queue:
        # Gets the first path in the queue
        path = queue.pop(0)

        # Gets the last node in the path
        vertex = path[-1]

        # Checks if we got to the end
        if vertex == end:
            return path
        # We check if the current node is already in the visited nodes set in order not to recheck it
        elif vertex not in visited:
            # enumerate all adjacent nodes, construct a new path and push it into the queue
            for current_neighbour in graph_to_search.get(vertex, []):
                new_path = list(path)
                new_path.append(current_neighbour)
                queue.append(new_path)

            # Mark the vertex as visited
            visited.add(vertex)

def part2(puzzle_input):
    data, _, goal = read_puzzle_input(puzzle_input)

    graph = {}
    for ii in range(data.shape[0]):
        for jj in range(data.shape[1]):
            curpos = (ii, jj)
            graph.setdefault((ii, jj), []).extend([ x[0] for x in adjacents(data, curpos) ])

    min_path = None
    for start in np.argwhere(data == 0):
        start = tuple(start)

        #pprint.pprint(graph)
        path = bfs(graph, start, goal)
        if path is None:
            #print(f"No path from {start} to {goal}")
            continue
        else:
            path_len = len(bfs(graph, start, goal))
            if min_path is None or path_len < min_path:
                min_path = path_len

    return min_path - 1

def test_part2():
    min_path = part2(os.path.join("..", "dat", "day12_test.txt"))
    assert min_path == 29
       
min_path = part2(os.path.join("..", "dat", "day12.txt"))
print(min_path)

430
======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-7.2.0, pluggy-1.0.0
rootdir: c:\Users\MichaelIhde\Git\advent-of-code-2022\python
collected 1 item

t_58129cfc90f04738840380712e605c16.py .                                                      [100%]

======================================== 1 passed in 0.01s ========================================
